In [110]:
import pandas as pd
from collections import Counter

import numpy as np
import random
import math

In [111]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [112]:
pwd

'/content'

In [113]:
ls

drive/  sample_data/


In [114]:
train_data = pd.read_csv('/content/drive/My Drive/LOCS/combined-swell-classification-eda-train-dataset.csv')
test_data = pd.read_csv('/content/drive/My Drive/LOCS/combined-swell-classification-eda-test-dataset.csv')
val_data = pd.read_csv('/content/drive/My Drive/LOCS/combined-swell-classification-eda-validation-dataset.csv')

train_X = train_data
test_X = test_data
val_X = val_data

# train_Y, test_Y, val_Y
train_Y = train_X["condition"]
test_Y = test_X["condition"]
val_Y = val_X["condition"]

In [115]:
train_X = train_X.drop(columns=['NasaTLX class', 'Condition Label', 'NasaTLX Label', 'condition', 'subject_id'])
test_X = test_X.drop(columns=['NasaTLX class', 'Condition Label', 'NasaTLX Label', 'condition', 'subject_id'])
val_X = val_X.drop(columns=['NasaTLX class', 'Condition Label', 'NasaTLX Label', 'condition', 'subject_id'])

In [116]:
class Node:
    def __init__(self):
        self.split_feature = None
        self.split_point = None
        self.result = None
        self.childs = None
        

In [117]:
class DecisionTree:
    
    def __init__(self):
        self.root = None
        
    def DT_print(self, cur_node = None, cnt = 0):
        if cnt == 0:
            cur_node = self.root
        print(' ' * cnt, "Level ", cnt," :: ", cur_node.split_feature, cur_node.split_point, cur_node.result)
        if cur_node.childs is None:
            return
        for child in cur_node.childs:
            self.DT_print(child, cnt + 1)
        
    def Gini_Impurity(self, data_Y):  # input label dataset of a group
        impurity = 1
        label_counts = Counter(data_Y)
        for label in label_counts:
            p_of_label = label_counts[label] / len(data_Y)
            impurity -= p_of_label ** 2
        return impurity
    
    def Information_Gain(self, unsplited_data_Y, splited_data_Y):
        gain = self.Gini_Impurity(unsplited_data_Y)
        # print(gain)
        # print(splited_data_Y)
        for subset in splited_data_Y:
            # print("-", Gini_Impurity(subset), " X ( ", len(subset), " / ", len(unsplited_data_Y), " )")
            gain -= self.Gini_Impurity(subset) * (len(subset)/ len(unsplited_data_Y))
        return gain
    
    def Split(self, data_X, data_Y, column):
        data_X_subsets = []    # 분할 후의 data_X 그룹을 저장하는 배열
        data_Y_subsets = []    # 분할 후의 data_Y 그룹을 저장하는 배열
        split_point = 0.0      # 최적 분할 기준 값
        split_point_gain = 0.0  # 최적 분할 지점에서의 Information Gain
        '''
        print("=-=- Before reset Index =-=-=-")
        print("**** data_X ****")
        print(data_X[column])
        print("**** data_Y ****")
        print(data_Y)
        '''
        data_X = data_X.sort_values(by=column)
        data_Y = data_Y[data_X.index]
        data_X = data_X.reset_index(drop=True)
        data_Y = data_Y.reset_index(drop=True)
        '''
        print("=-=- After reset Index =-=-=-")
        print(data_X.index)
        print(data_X[column])
        print(data_Y)

        print(data_X[column].iloc[5], data_Y[5])
        '''
        for i in range(1, len(data_Y)):
            candidate_splited_data_X = []
            candidate_splited_data_Y = []
            if data_Y[i-1] != data_Y[i]:
                # print(i, data_Y[i-1], data_Y[i])
                candidate_point = (data_X[column].iloc[i-1] + data_X[column].iloc[i]) / 2
                candidate_splited_data_Y.append(data_Y[:i])
                candidate_splited_data_Y.append(data_Y[i:])
                gain = self.Information_Gain(data_Y, candidate_splited_data_Y)
                if gain > split_point_gain:
                    candidate_splited_data_X.append(data_X[:i])
                    candidate_splited_data_X.append(data_X[i:])
                    split_point = candidate_point
                    split_point_gain = gain
                    data_X_subsets = candidate_splited_data_X
                    data_Y_subsets = candidate_splited_data_Y
                    # print("== Updated :: ", split_point_gain, split_point)
        return split_point_gain, split_point, data_X_subsets, data_Y_subsets
    
    def Find_Best_Split(self, data_X, data_Y, bagging=False):
        # print("=-=-New Group=-=-")
        best_feature = ''    # 데이터를 분할 할 feature
        best_gain = 0.0       # 데이터를 특정 feature로 분할했을 때 가장 높게 측정된 Information Gain
        best_split_point = 0.0

        if bagging:
          num_feature = math.ceil(math.sqrt(len(data_X.columns)))
          columns = np.random.choice(len(data_X.columns), num_feature, replace=False)
          for col_idx in columns:
            column = data_X.columns[col_idx]
            gain, split_point = self.Split(data_X, data_Y, column)[0:2]
            if gain > best_gain:
                best_gain = gain
                best_feature = column
                best_split_point = split_point
          return best_feature, best_gain, best_split_point

        for column in data_X.columns:  # RF에서 Bagging Features 적용 필요.
            # print("check column :: ", column)
            gain, split_point = self.Split(data_X, data_Y, column)[0:2]
            if gain > best_gain:
                best_gain = gain
                best_feature = column
                best_split_point = split_point
        return best_feature, best_gain, best_split_point
    
    def fit(self, data_X, data_Y, cnt=0, bagging=False):
        root = Node()

        data_X = data_X.reset_index(drop=True)
        data_Y = data_Y.reset_index(drop=True)

        best_feature, best_gain, best_split_point = self.Find_Best_Split(data_X, data_Y, bagging)
        if best_gain == 0:
            root.result = data_Y[0]
            # print(' ' * cnt, "== No Split ", cnt," :: ", root.result)
            return root
        data_X_subsets, data_Y_subsets = self.Split(data_X, data_Y, best_feature)[2:]

        # print(' ' * cnt, "== Split ", cnt," :: ", best_feature, best_gain)
        childs = []
        for i in range(len(data_X_subsets)):
            childs.append(self.fit(data_X_subsets[i], data_Y_subsets[i], cnt+1))

        root.split_feature = best_feature
        root.split_point = best_split_point
        root.childs = childs
        if cnt == 0:
            self.root = root
        return root
    
    def predict(self, dataset):
        result = []
        for i in dataset.index:
            cur_node = self.root
            while cur_node.result is None:
                value = dataset.loc[i, cur_node.split_feature]
                if value < cur_node.split_point:
                    cur_node = cur_node.childs[0]
                else :
                    cur_node = cur_node.childs[1]
            result.append(cur_node.result)
        return result
    
    def score(self, data_X, data_Y):
        predict = self.predict(data_X)
        cnt = 0
        for i in range(len(data_Y)):
            if predict[i] == data_Y[i]:
                cnt += 1
                
        return cnt / len(data_Y)

In [120]:
newDT = DecisionTree()
newDT.fit(train_X.head(100), train_Y.head(100))

In [121]:
newDT.score(val_X.head(100), val_Y.head(100))

0.5

In [122]:
class RandomForest:
    def __init__(self):
        self.trees = []
        self.result = None

    def bagging_data(self, data_X, data_Y):
        indices = [random.randint(0, len(data_X)-1) for x in range(len(data_X))]
        data_X_subset = data_X.iloc[indices]
        data_Y_subset = data_Y.iloc[indices]
        data_X_subset = data_X_subset.reset_index(drop=True)
        data_Y_subset = data_Y_subset.reset_index(drop=True)
        return data_X_subset, data_Y_subset

    def fit(self, data_X, data_Y, tree_num = 10):
        self.trees = []
        for i in range(tree_num):
          data_X_subset, data_Y_subset = self.bagging_data(data_X, data_Y)
          tree = DecisionTree()
          tree.fit(data_X_subset, data_Y_subset, bagging=True)
          self.trees.append(tree)

    def predict(self, dataset):
        results = []
        for tree in self.trees:
          result = tree.predict(dataset)
          results.append(result)
        results = np.transpose(results)
        return results

In [123]:
RF = RandomForest()

In [124]:
RF.fit(train_X.head(10), train_Y.head(10))
RF.trees